In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup as BS
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager(driver_version="131.0.6778.70").install()),options=options)

## 스펙 조회

In [231]:
url = "https://auto.danawa.com/auto/?Work=model&Model=1542"

driver.get(url)

bs = BS(driver.page_source)

In [227]:
# 시세
import re

min_price = 0
max_price = 0

price = model_info.find('div', 'newcar').find('div', 'price_title').text.split("~")

re_price = [re.sub(r'[^0-9]', '', x) for x in price]

if len(re_price) == 1:
    min_price = re_price
    
else:
    min_price = re_price[0]
    max_price = re_price[1]

In [237]:
# 스펙
spec = []
spec_list = []
car_size = None
fuel_type = None
cc = None
fuel_efficiency = None
etc = None

for x in bs.find('div', 'spec').findAll('span'):
    if x.find('span') == None:
        spec.append(x.text)

car_size = spec[0]
fuel_type = spec[1]
spec = spec[2:]

for x in spec:
    if 'cc' in x:
        cc = x
        spec.remove(x)
        break
    else:
        cc = None
for x in spec:
    if ('연비' in x) or ('전비' in x):
        fuel_efficiency = x
        spec.remove(x)
        break
    else:
        fuel_efficiency = None
        
if len(spec) > 0:
    etc = (", ").join(spec)
else:
    etc = None
    
spec_list = [car_size, fuel_type, cc, fuel_efficiency, etc]

['중형', '연비인증中', None, None, None]

## 브랜드 코드 조회

In [252]:
# 브랜드 코드 조회

url = "https://auto.danawa.com/auto/?Work=home"

driver.get(url)

from bs4 import BeautifulSoup as BS

bs3 = BS(driver.page_source)

# 국내 브랜드 리스트
brand_list_kr = [ x['href'] for x in bs3.find_all('ul', 'brandList')[0].find_all('a') ]

brand_code_kr = [ x.split('=')[-1] for x in brand_list_kr ]

In [253]:
brand_list_kr = [ x['href'] for x in bs3.find_all('ul', 'brandList')[0].find_all('a') ]

In [254]:
brand_code_kr = [ x.split('=')[-1] for x in brand_list_kr ]

In [255]:
# 국내 브랜드 코드 --> 현대, 기아, 제네시스, 쉐보레, 르노 삼성
brand_cd_kr = brand_code_kr[ : 5]

## 모델코드 조회

In [134]:
url = f"https://auto.danawa.com/auto/?Work=brand&Brand={brand_code_kr[1]}"

driver.get(url)

from bs4 import BeautifulSoup as BS

brand_home = BS(driver.page_source)

# 모델코드
brand_model_code = [ x['code'] for x in bs2.find('dl', 'modelBox').find_all('li') ]

## 전체 데이터 생성 코드

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup as BS
from tqdm import tqdm
import time
import re


options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager(driver_version="131.0.6778.70").install()),options=options)


total_spec = []

for x in tqdm(brand_cd_br):
    url = f"https://auto.danawa.com/auto/?Work=brand&Brand={x}"
    
    driver.get(url)
    
    time.sleep(2)
    
    brand_info = BS(driver.page_source)

    # 모델 코드
    try:
        model_code = [ x1['code'] for x1 in brand_info.find('dl', 'modelBox').find_all('li') ]
    except Exception as e:
        print(e)
    
    for y in tqdm(model_code):
        try:
            url = f"https://auto.danawa.com/auto/?Work=model&Model={y}"

            driver.get(url)
            
            time.sleep(2)
            
            model_info = BS(driver.page_source)
            
            # 시세
            min_price = 0
            max_price = 0
            
            price = model_info.find('div', 'newcar').find('div', 'price_title').text.split("~")
            
            re_price = [re.sub(r'[^0-9]', '', x) for x in price]
            
            if len(re_price) > 1:
                min_price = re_price[0]
                max_price = re_price[1]
            elif (len(re_price) == 1) & (re_price[0] != ''):
                min_price = re_price[0]
                max_price = 0
            else:
                min_price = 0
                max_price = 0
            
            # 스펙
            spec = []
            spec_list = []
            car_size = None
            fuel_type = None
            cc = None
            fuel_efficiency = None
            etc = None
            
            for x in model_info.find('div', 'spec').findAll('span'):
                if x.find('span') == None:
                    spec.append(x.text)
            
            car_size = spec[0]
            fuel_type = spec[1]
            spec = spec[2:]
            
            for x in spec:
                if 'cc' in x:
                    cc = x
                    spec.remove(x)
                    break
                else:
                    cc = None
            for x in spec:
                if ('연비' in x) or ('전비' in x):
                    fuel_efficiency = x
                    spec.remove(x)
                    break
                else:
                    fuel_efficiency = None
                    
            if len(spec) > 0:
                etc = (", ").join(spec)
            else:
                etc = None
                
            spec_list = [y, min_price, max_price, car_size, fuel_type, cc, fuel_efficiency, etc]

            total_spec.append(spec_list)
        except Exception as e:
            print(e)

 71%|██████████████████████████████████████████████████████████▊                        | 17/24 [00:48<00:18,  2.65s/it]

## DB Table 생성

In [282]:
import pandas as pd

df_info_kr = pd.DataFrame(total_spec, columns= ['cd_model','min_price','max_price','car_size','fuel_type','cc','fuel_efficiency', 'etc'])

In [284]:
import sqlalchemy
from urllib import parse

user='team2'
password='Encore_team2@'
password = parse.quote_plus(password)
host='192.168.0.95'
#host='222.112.208.67'
port=3306
database='team2'
engine=sqlalchemy.create_engine(f'mysql://{user}:{password}@{host}:{port}/{database}')

df_info_kr.to_sql('danawa_carInfo', if_exists='replace', con=engine, index=False)

88